Librerie utili 


In [1]:
import pandas as pd
import requests #Pushshift accede a reddit attraverso un URL 
import json 
import csv 
import time
import datetime

Viene definita una funzione chiamata getData, la quale costruisce un Pushfit URL, accede alla webpage e archivia tutti i dati JSON e ritorna l'elemento data che contiene tutti i dati relativi ad una submission

In [7]:
list
def getData(query1,query2, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query1)+'|'+str(query2)+'&size=100&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

Viene definita la funzione collectData che ha il compito di estrarre i principali dati da ogni risultato JSON ottenuto dalla funzione getData, i quali vengono prima inseriti in una lista per poi venire memorizzati in un dizionario che ha come chiave il sub_id (identificativo univoco della submission)

In [8]:

def collectData(subm):
    
    subData = list() 
    title = subm['title']
    url = subm['url']
    #try:
       # body=subm['selftext']
    #except KeyError:
        #body='Nan'
    
       
    #flairs are not always present so we wrap in try/except
    #try:
        #flair = subm['link_flair_text']
    #except KeyError:
        #flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) # la data viene trasformata da timestamp in datetime
    numComms = subm['num_comments']
    subData.append((sub_id,title,url,author,score,created,numComms))
    subStats[sub_id] = subData

Definire il periodo temporale d'interesse, le parole chiave da ricercare e il subreddit in cui cercare. Viene poi definita una variabile contatore che permette di contare il numero totale di submission che abbiamo raccolta e viene definito il dizionario in cui verranno conservati i dati

In [9]:
beginning_timestamp = int(datetime.datetime(year=2021, month=1, day=11).timestamp())
end_timestamp = int(datetime.datetime(year=2021, month=2, day=4).timestamp())
after = beginning_timestamp
before = end_timestamp
query1 = "GME" # le query non sono case-sensitive
query2='GameStop'

sub = "wallstreetbets" 
subCount = 0
subStats = {}

Questo script permette di fare più richieste all'api di Reddit attraverso la funzione GetData, fichè il valore di after e before non diventano lo stesso. Per ogni richiesta la data viene aggiornata con la data dell'ultima submission scaricata dalla precedente richiesta.

Viene dato un po' di context per vedere meglio effettivamente se quello che è stato scaricato è giusto

In [12]:
print(str(len(subStats)) + " submissions aggiunte alla lista")
print("La prima submission è:")
print(list(subStats.values())[0][0][1] + " creata: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " creata: " + str(list(subStats.values())[-1][0][5]))

1100 submissions aggiunte alla lista
La prima submission è:
70% SPIKE, gme 🚀🚀🚀🚀 creata: 2021-01-11 00:19:52
Last entry is:
My Hinge Dating Match 😳 works in Finances 📈 and her DD says GME 🎮 is going higher! 🚀🚀🚀 Throw some moon rocks at those gay bears!!! 🌈🐻 Diamonds Hands ! 💎👐 creata: 2021-01-14 08:47:32


Tramite questa funzione i dati collezionati nel dizionario in un formato JSON vengono trasformati in un dataset in formato csv

In [ ]:
def subtocsv():
    upload_count = 0
    print("input filename of submission file, please add .csv")
    filename = input() # viene chiesto all'utenete di nominare il file
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Url","Body","Author","Score","Publish Date","Total No. of Comments"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
subtocsv()